In [26]:
from utils import *
from external_packages import *
import torch

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr_torch.models import DeepFM
from deepctr_torch.inputs import  SparseFeat, DenseFeat,get_fixlen_feature_names

In [4]:
final_stackset=pd.read_csv(processed_data_folder+'/final_stackset_PowerTransformer.csv',index_col=0)
final_stackset_features=final_stackset.drop(['ID','to_predict'],axis=1)
final_stackset_target=final_stackset.loc[:,['to_predict']]
final_stackset_target.groupby('to_predict')['to_predict'].count()

to_predict
0    999756
1       244
Name: to_predict, dtype: int64

In [77]:
dense_features = final_stackset.columns[1:]

In [78]:
final_stackset[dense_features] = final_stackset[dense_features].fillna(0,)

In [79]:
target = ['to_predict']

In [80]:
dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

In [81]:
feature_names=get_fixlen_feature_names(dense_feature_columns)

In [82]:
'to_predict' in feature_names

True

In [83]:
'ID' in feature_names

False

In [84]:
train, test =train_test_split(final_stackset[feature_names],
                        test_size=0.2,random_state =random_state )

In [85]:
train_model_input = [train[name] for name in feature_names]

test_model_input = [test[name] for name in feature_names]

In [88]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(dense_feature_columns,dense_feature_columns,task='binary',device=device)
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.1, )
pred_ans = model.predict(test_model_input, batch_size=256)

cuda ready...
[(720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,), (720000,)]
(720000, 69)
cuda:0
Train on 720000 samples, validate on 80000 samples
torch.Size([256, 69])


ValueError: y_true contains only one label (0.0). Please provide the true labels explicitly through the labels argument.